In [ ]:
Importing Important Libraries
Steps To Be Followed
Importing necessary Libraries
Creating S3 bucket
Mapping train And Test Data in S3
Mapping The path of the models in S3

In [49]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

In [53]:
bucket_name = 'cris2021bankapp' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)
print(bucket_name)

us-east-1
cris2021bankapp


In [14]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [15]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix) # first {} is for bucket name and second {} is for prefix
print(output_path)

s3://cris2021bankapp/xgboost-as-a-built-in-algo/output


In [33]:

import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)
    
#try:
    #model_data = pd.read_excel("https://cris2021bankapp.s3.amazonaws.com/megasena.xlsx")#, sheet_name="loterica")
  #  model_data = pd.read_excel('https://cris2021bankapp.s3.amazonaws.com/megaxls.xlsx')
 #   print(model_data.head(10))
    
#except Exception as e:
 #   print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [36]:
ls

bank_clean.csv  CrisMLBankDemo.ipynb  lost+found/  train.csv


In [34]:

### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [35]:
train_data

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
40949,54,3,999,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
9332,56,2,999,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0
32286,32,2,999,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0
3925,46,3,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
9406,35,2,999,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3871,32,2,999,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0
16681,26,2,999,0,1,0,0,1,0,0,...,0,0,0,0,1,0,1,0,1,0
39272,33,1,6,2,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,1
7717,32,4,999,0,1,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,1


In [38]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
#s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')


In [39]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [56]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = sagemaker.image_uris.retrieve('xgboost', my_region, 'latest')
#container = sagemaker.image_uris.retrieve(boto3.Session().region_name,'xgboost', repo_version='1.0-1')

#container = sagemaker.image_uris.retrieve(boto3.Session().region_name,'xgboost', repo_version='1.0-1')


In [57]:

# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [62]:

# construct a SageMaker estimator that calls the xgboost-container
#estimator = sagemaker.estimator.Estimator(container, 
 #                                         hyperparameters=hyperparameters,
  #                                        role=sagemaker.get_execution_role(),
   #                                       train_instance_count=1, 
    #                                      train_instance_type='ml.m5.2xlarge', 
     #                                     train_volume_size=5, # 5 GB 
      #                                    output_path=output_path,
       #                                   train_use_spot_instances=True,
        #                                  train_max_run=300,
         #                                 train_max_wait=600)

estimator = sagemaker.estimator.Estimator(container,
                                          sagemaker.get_execution_role(),
                                          hyperparameters=hyperparameters,
                                          instance_count=1,
                                          instance_type='ml.m5.2xlarge',
                                          volume_size=5,
                                          max_run=300,
                                          max_wait=600,
                                          use_spot_instances=True,
                                          input_mode='File',
                                          output_path=output_path)

In [63]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2021-03-11 19:53:19 Starting - Starting the training job...
2021-03-11 19:53:33 Starting - Launching requested ML instancesProfilerReport-1615492399: InProgress
......
2021-03-11 19:54:45 Starting - Preparing the instances for training......
2021-03-11 19:55:44 Downloading - Downloading input data
2021-03-11 19:55:44 Training - Downloading the training image..Arguments: train
[2021-03-11:19:56:00:INFO] Running standalone xgboost training.
[2021-03-11:19:56:00:INFO] File size need to be processed in the node: 4.83mb. Available memory size in the node: 23849.56mb
[2021-03-11:19:56:00:INFO] Determined delimiter of CSV input is ','
[19:56:00] S3DistributionType set as FullyReplicated
[19:56:00] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-03-11:19:56:00:INFO] Determined delimiter of CSV input is ','
[19:56:00] S3DistributionType set as FullyReplicated
[19:56:00] 12357x59 matrix with 729063 entries loaded from /opt/ml/